# average traces

In [ ]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
%matplotlib widget
import mmap
import dask.array as da

from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer


In [ ]:
folder_base = Path('/Users/ahay/Documents/DATA/OPPK_Annie/2023-09-27_15-02-22/RecordNode116/experiment1/recording1/continuous/Acquisition_Board-115RhythmData/')

filename = folder_base / f'continuous.dat'
All = np.fromfile(filename, dtype="int16")
All = All.reshape(-1,64)
#All = np.load(filename2, mmap_mode= 'r')
#All = All.T

All = All[:, 0:32]
All.shape

Load timestamps and find the first of each series

In [ ]:
filenameT = folder_base / f'timestamps2.npy'
Timestamps = np.load(filenameT)
Timestamps = Timestamps*2000
Timestamps = Timestamps.astype(int)
Timestamps
delaystamp = 1000

In [ ]:
stamps_time = Timestamps
first_stamps = []
delay_stamps = []
number_stamps = len(stamps_time)
for i in range(number_stamps -1):
    delay_stamps.append(stamps_time[i+1] - stamps_time[i])
    if delay_stamps[i] > delaystamp:
        first_stamps.append(stamps_time[i])

In [ ]:
AllC0 = All[:,6]
AllC11 = All[:,22]-All[:,23]
AllC12 = All[:,24]-All[:,25]
AllC13 = All[:,26]-All[:,27]
AllC14 = All[:,28]-All[:,29]
AllC15 = All[:,20]-All[:,21]
L = [AllC0, AllC12, AllC13, AllC11, AllC14, AllC15]
All2 = np.concatenate(L, axis=0)
All2 = All2.reshape(6,-1)
All2 = All2.T
All2.shape

In [ ]:
numb_sweep = len(first_stamps)
TruncAll2 = np.zeros((2000, 6))
for i in range(numb_sweep):
    AllT = All2[first_stamps[i]-1200:first_stamps[i]+800,:]
    TruncAll2 = TruncAll2 + AllT

In [ ]:
# Defining subset
start = 0
end = 2000

times = np.arange(0, 2000/2000, 1./2000)
tt = times[start:end]
Cortext = TruncAll2[start:end, 4]

plt.close('all')
plt.plot(tt, Cortext)

plt.show()

filename = folder_base / f'M18S1.svg'
plt.savefig(filename, format="svg")

In [ ]:




app = mkQApp()


sample_rate = 2000.
t_start = 0.

TTL = Timestamps

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: TTL, 1: TTL}
#and asign them to some channels each
scatter_channels = {0: [0, 2], 1: [0, 1]}
source = AnalogSignalSourceWithScatter(All2, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel

view1.by_channel_params['ch0', 'gain'] = 0.0000002
view1.by_channel_params['ch1', 'gain'] = 0.00002
view1.by_channel_params['ch2', 'gain'] = 0.00002
view1.by_channel_params['ch3', 'gain'] = 0.00002
view1.by_channel_params['ch4', 'gain'] = 0.00002

view1.by_channel_params['ch0', 'offset'] = 0.2
view1.by_channel_params['ch1', 'offset'] = 0.1
view1.by_channel_params['ch2', 'offset'] = 0.0
view1.by_channel_params['ch3', 'offset'] = -0.1
view1.by_channel_params['ch4', 'offset'] = -0.2



#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()